In [919]:
import numpy as np
import os 

In [920]:
def lu_decomposition(A):
    n = A.shape[0]
    L = np.zeros((n, n))
    U = np.zeros((n, n))  
    for i in range(n):
        L[i, i] = 1.0
        for j in range(i, n):
            U[i, j] = A[i, j] - np.dot(L[i, :i], U[:i, j])
        for j in range(i+1, n):
            L[j, i] = (A[j, i] - np.dot(L[j, :i], U[:i, i])) / U[i, i]
    return L, U
def solve_lu(A, b):
    L, U = lu_decomposition(A)
    n = A.shape[0]
    y = np.zeros((n, 1))
    x = np.zeros((n, 1))

    # Solve Ly = b using forward substitution
    for i in range(n):
        y[i, 0] = b[i, 0] - np.dot(L[i, :i], y[:i, 0])

    # Solve Ux = y using backward substitution
    for i in range(n - 1, -1, -1):
        x[i, 0] = (y[i, 0] - np.dot(U[i, i+1:], x[i+1:, 0])) / U[i, i]
    return x

In [921]:
def solve_linear_system(A, b):
    try:
        # Attempt to solve the system using numpy.linalg.solve()
        X = np.linalg.solve(A, b)
        return X
    except np.linalg.LinAlgError:
        # Handle the case where A is singular (not invertible)
        raise ValueError("Matrix A is singular, and the system has no unique solution.")

In [922]:
def get_abc(Grid_points,rho,dt,dx,f,A,u,p_s):
    b=np.ones((Grid_points))
    a=np.ones((Grid_points))
    c=np.ones((Grid_points-1))

    for i in range (0,Grid_points):
        a[i]=((rho*A[2*i]/dt)+(f*rho*u[i]*p_s[2*i]/8))
        b[i]=((rho*A[2*i]/(2*dx)))
        if i<Grid_points-1:
            c[i]=(A[(2*i)+1]/dx)             
    return a,b,c        
def get_d(Grid_points,rho,A,u,dt):
    d=np.ones((Grid_points))
    for i in range(0,Grid_points):
        d[i]=((rho*A[2*i]*u[i]/dt))  
    return d             

In [923]:
#for momentum equation final draft
def solve_momentum(a, b, c, d, p_star,Grid_points,u_initial,u_final):
    """
    Calculate u_star vector using the provided inputs and matrix inversion.

    Parameters:
    a (numpy.ndarray): Coefficients a_{i+1/2}^{n+1}.
    b (numpy.ndarray): Coefficients b_{i-1/2}^{n+1} & b_{i+3/2}^{n+1}.
    c (numpy.ndarray): Coefficients c_i^{n+1}.
    d (numpy.ndarray): Coefficients d_{i+1/2}^{n}.
    p_star (numpy.ndarray): Guessed pressure vector.

    Returns:
    u_star (numpy.ndarray): Calculated u_star vector.
    """
    #len of p_star is equal to length of a-1;
    # Calculate the size of the system (number of equations)
    n =Grid_points
    if n>2:
        # Initialize the coefficient matrix A and the right-hand side vector b
        A = np.zeros((n-2, n))
        u_star=np.zeros((n,1))
        Rhs=np.zeros((n-2,1))
        # Populate the coefficient matrix and the right-hand side vector
        for i in range(1,n-1):
            A[i-1, i-1] =b[i-1]
            A[i-1,i] = (-1*a[i])
            A[i-1,i+1]=b[i+1]
            const = ((c[i] * p_star[i]) - (c[i-1] * p_star[i-1]) - d[i])
            Rhs[i-1,0]=const
        #print(Rhs[0,0],Rhs[n-3,0])
        #print(A,Rhs) 
        Rhs[0,0]= Rhs[0,0]-(A[0,0]*u_initial)
        Rhs[n-3,0]=Rhs[n-3,0]-(A[n-3,n-1]*u_final)
        #print(Rhs[0,0],Rhs[n-3,0])     
        coefficient_matrix=A[:,1:n-1]
        #print(coefficient_matrix,Rhs)       
        u_star=solve_lu(coefficient_matrix,Rhs)
        #print(solve_linear_system(coefficient_matrix,Rhs))

    return u_star    

In [924]:
#pressure correction coefficients:
def get(Grid_points,u,u_star,A,p_s,rho,dx,dt,f):
    alpha=np.zeros(Grid_points-1)
    beta=np.zeros(Grid_points-1)
    gama=np.zeros(Grid_points-1)
    z=np.zeros(Grid_points-1)
    if (Grid_points>1): 
        for i in range(1,(2*Grid_points)-2,2):
            a_left=(rho*A[i-1]/dx)
            b_left=(rho*A[i-1]/dt)
            c_left=(f*rho*u[(i-1)//2]*p_s[i-1]/8) //2
            a_right=(rho*A[i+1]/dx)
            b_right=(rho*A[i+1]/dt)
            c_right=(f*rho*u[(i+1)//2]*p_s[i+1]/8)
            beta1_left=(a_left/(b_left+c_left))
            gama1_right=(a_right/(b_right+c_right))

            alpha[(i-1)//2]=(beta1_left+gama1_right)*(A[i]/dx)
            beta[(i-1)//2]=beta1_left*(A[i]/dx)
            gama[(i-1)//2]=gama1_right*(A[i]/dx)
            z[(i-1)//2]=((a_left*(u_star[(i-1)//2]))-(a_right*(u_star[(i+1)//2]))+(((rho*A[i])-(rho*A[i]))/dt))
    return alpha,gama,beta,z


In [925]:
def Combine(u_star1,u_star,Grid_points):
    u_star[1:Grid_points-1]=u_star1[:, 0]
    return u_star

In [926]:
def add(p_star,add_p,Grid_points):
    p_star[1:Grid_points-2] += add_p[:,0]
    return p_star

In [927]:
def correct_pressure(Grid_points,alpha,beta,gama,z,p_entry,p_exit):
    """
    Calculate P' vector using the provided inputs and matrix inversion.

    Parameters:
    alpha (numpy.ndarray):
    beta (numpy.ndarray): 
    gama (numpy.ndarray):
    z(numpy.ndarray):

    Returns:
    p' (numpy.ndarray): Calculated p'(It is the value of p that has to be added) vector.
    """
    n=Grid_points
    A1 = np.zeros((n-3, n-1))
    Rhs1=np.zeros((n-3,1))
    #populate the coefficent matrix
    for i in range(1,n-2):
        A1[i-1,i-1] = gama[i-1]
        A1[i-1,i] = (-1*alpha[i-1])
        A1[i-1,i+1]= (beta[i-1])
        const = (-1*z[i-1])
        Rhs1[i-1,0]=const
    #print(A1,Rhs1)    
    Rhs1[0,0]=Rhs1[0,0]-(A1[0,0]*p_entry)
    Rhs1[n-4,0]=Rhs1[n-4,0]-(A1[n-4,n-2]*p_exit)
    co_efficientmatrix1=A1[:,1:n-2] 
    #print(co_efficientmatrix1,Rhs1)
    p_=solve_lu(co_efficientmatrix1,Rhs1)    
    return p_

    
    

In [928]:
# main algo
def unsteady_1D_flow(A,u,p_star,u_star,p_s,Grid_points,rho,dx,dt,f,n):
    for t in range(0,n):
        """    
        #Mometum equation part 
        """
        a_values,b_values,c_values=get_abc(Grid_points,rho,dt,dx,f,A,u,p_s)#solve co-efficients at all grid points
        d_values=get_d(Grid_points,rho,A,u,dt)
        #define Boubdary conditions for velocity
        u_inlet=4
        u_star[0]=u_inlet
        u_exit=4
        u_star[Grid_points-1]=u_exit
        u_star1= solve_momentum(a_values, b_values, c_values, d_values, p_star,Grid_points,u_inlet,u_exit)# Solve momentum equation to find intermediate velocity (u_star)
        #print(u_star,u_star1)
        u_star=Combine(u_star1,u_star,Grid_points)
        #print(u_star)
    
        """    
        #Pressure Adjustment part 
        """    
        alpha,beta,gama,z=get(Grid_points,u,u_star,A,p_s,rho,dx,dt,f) 
        #print(alpha,beta,gama,z)
        #print(alpha.size)
        p_inlet=1
        p_exit=1
        p_star[0]=p_inlet
        p_star[Grid_points-2]
        add_p1=correct_pressure(Grid_points,alpha,beta,gama,z,p_inlet,p_exit)#Solve pressure correction equation 
        print(add_p1)
        p_updated=add(p_star,add_p1,Grid_points)
        print(p_updated)






    return u_star,p_s

In [929]:
# Define constants and parameters
n = 1  # Number of time steps
L = 1.0  # Length of the 1D domain
dx = 1 # Spatial grid size
dt = 1  # Time step size
rho = 1.0 # Density
A = 1.0 # Cross-sectional area
f = 1.0   # Source term
p_s = 1.0  # Viscosity
g = 10.0   # Gravitational acceleration
Grid_points=5 #Total number of grid points 

# Initialize arrays for velocity (u) and pressure (p) at staggered grid locations
u = np.ones(Grid_points)  # Velocity at staggered grid locations
p_star = np.ones(Grid_points-1) # Pressure at cell centers
Area=np.ones((2*Grid_points)-1)
p_s=np.ones((2*Grid_points)-1)
u_star= np.ones(Grid_points)
velocity, Pressure =unsteady_1D_flow(Area,u,p_star,u_star,p_s,Grid_points,rho,dx,dt,f,n)









[[0.14342142]
 [0.37364808]]
[1.         1.14342142 1.37364808 1.        ]
